In [3]:
import random 
import simpy
import numpy as np
import pandas as pd

#### This code sets up a simulation of a call center, where customers enter a waiting queue, request support, and leave the call after receiving support. 
#### The simulation runs for a specified duration, and at the end, it provides the count of customers served.

### Challenge;
###  Implement a call center with multiple priority levels for customer support.

In [11]:
NUM_EMPLOYEES = 2
AVG_SUPPORT_TIME = 5 # time for a call in min
CUSTOMER_CALL_INTERVAL = 2 # min
SIMULATION_TIME = 120 # min


customers_served = 0

class CallCenter:
    def __init__(self, env, num_employees, support_time):
        self.env = env
        self.staff= simpy.Resource(env, num_employees)
        self.support_time = support_time

    #Simulate support time
    def support(self, customer):
        random_support_time = max(1, np.random.normal(self.support_time, 4))    
        yield self.env.timeout(random_support_time)   #waiting time
        #print(f"Support finished for {customer} at {self.env.now:.2f}")

#simulate customers
def customer(env, name , call_center: CallCenter, priority, data):
    global customers_served
    #print(f"Customer{name} enters waiting queue at {env.now:.2f}!")
    with call_center.staff.request() as request:
        yield request
       # print(f"Customer {name} enters call at {env.now:.2f}")
        start_time = env.now
        yield env.process(call_center.support(name))
        end_time = env.now
       # print(f"Customer {name} left call at {env.now:.2f}")
        customers_served += 1
        data.append({"Customer": name, "Priority": priority,"Service Duration": end_time - start_time})


def setup(env, num_employees, support_time, customer_interval, data):
    call_center = CallCenter(env, num_employees, support_time)     
    for i in range(1, 6):
        env.process(customer(env, i, call_center, priority=random.choice(["HIGH", "LOW"]), data=data))

    while True:
        yield env.timeout(random.randint(customer_interval - 1, customer_interval + 1))       
        i = i + 1
        env.process(customer(env, i, call_center,priority=random.choice(["HIGH", "LOW"]),data=data))

print("Starting call center Simulation")
env = simpy.Environment()
customer_data = []
env.process(setup(env, NUM_EMPLOYEES, AVG_SUPPORT_TIME,CUSTOMER_CALL_INTERVAL, data = customer_data))
env.run(until=SIMULATION_TIME)

#print("Customer handled: ", customers_served)
df = pd.DataFrame(customer_data)
df.head(20)


Starting call center Simulation


,Customer,Priority,Service Duration
0,2,HIGH,5.160280
1,1,HIGH,8.477805
2,3,LOW,8.615097
3,4,HIGH,9.443616
4,5,LOW,9.972203
5,6,HIGH,9.434962
6,7,LOW,5.565691
7,9,LOW,1.563500
8,8,HIGH,4.427764
9,11,HIGH,9.152410
